In [1]:
import lib
import numpy as np

# Hartree Fock computation of the ground state of the Helium atom

In this Jupyter Notebook we are going to compare the groundstate energies obtained with the analytical formula for the two body integrals with Montecarlo integration. In both cases, the Self-Consistent Field method is used.

### Basis functions
First we write the basis we are going to use

In [3]:
ALPHA_1 =  0.298073
ALPHA_2 =  1.242567
ALPHA_3 =  5.782948
ALPHA_4 = 38.474970
ALPHA = [ALPHA_1, ALPHA_2, ALPHA_3,ALPHA_4]

def He_wf(x, y, z, n):
    """
    Evaluates the n^th eigenfunction of the Hydrogen atom with exponential coefficient alpha_i. 
    The units of x are atomic units. 

    Parameters
    ==========
    x, y, z : float or np.ndarray
        Position in which to evaluate the wave function
    n : int
        Number of the wave function

    Returns
    =======
    float or np.ndarray
    """

    if n == 1:
        alpha = ALPHA_1
    elif n == 2:
        alpha = ALPHA_2
    elif n == 3:
        alpha = ALPHA_3
    elif n == 4:
        alpha = ALPHA_4
    else:
        alpha = None

    r2 = x**2 + y**2 + z**2

    return np.exp(-alpha*r2)

def He_wf_basis(R,k):
    """
    Evaluates the k^th eigenfunction of the Hydrogen atom with exponential coefficient alpha_i. 
    The units of x are atomic units. 

    Parameters
    ==========
    R : np.ndarray(3,N)
        Position in which to evaluate the wave function
    k : int
        Number of the wave function

    Returns
    =======
    float or np.ndarray(N)
    """
    return He_wf(R[0],R[1],R[2],k)

We now write the functions to calculate the integrands of the two-body integrals

In [4]:
def integrand_2(R):
    r1 = R[:,0:3]
    r2 = R[:,3:6]
    r12 = np.sqrt(np.sum((r1 - r2)**2, axis=-1))
    return 1/r12

def norm_product(p, r, q, s):
    a = ALPHA[p-1] + ALPHA[q-1]
    b = ALPHA[r-1] + ALPHA[s-1]
    PROD = (np.pi/a)**1.5 * (np.pi/b)**1.5
    
    return PROD

def He_two_body_integrand(p, r, q, s):
    """
    Integrand that goes into the monte carlo method in order to compute the 
    two electron integrals.

    Parameters
    ==========
    indices : int np.ndarray(4)
        Indices of the wave function

    Returns
    =======
    function
    """
    
    f = lambda R: integrand_2(R)*norm_product(p, r, q, s)
    
    return f

### One- and two- body integral computations
We first initialize some necessary parameters

In [5]:
N_electrons = 2
N_basis = 4
N_points = 1000000
integrals_file_num = "integrals_He_num.npy" # numerical integrals
integrals_file_ana = "integrals_He_ana.npy" # analytical integrals

normalized_wf = False

max_iter_SCF = 500
eps_SCF = 1E-5
Delta_SCF = 0

The exact one-body integral function is

In [6]:
def analytical_1(p,q):
    I = 3*ALPHA[p-1]*ALPHA[q-1]*np.pi**(1.5)/(ALPHA[p-1] + ALPHA[q-1])**(5/2) - 4*np.pi/(ALPHA[p-1] + ALPHA[q-1])
    return I

The exact two-body integral function is

In [7]:
def analytical_2(p, r, q, s):
    I = 2*np.pi**2.5 / ((ALPHA[p-1] + ALPHA[q-1])*(ALPHA[r-1] + ALPHA[s-1])*np.sqrt((ALPHA[p-1] + ALPHA[q-1] + ALPHA[r-1] + ALPHA[s-1])))
    return I

The covariance function for the exponents of the gaussians is

In [8]:
def f_cov(p, r, q, s):
    cov = 0.5*np.diag([1/(ALPHA[p-1] + ALPHA[q-1])]*3 + [1/(ALPHA[r-1] + ALPHA[s-1])]*3)
    return cov

We then calculate and store the one- and two-body integrals using lib.py

In [10]:
# One- and Two-body integrals (numerical)
MC_args = {"f_cov":f_cov, "f_integrand":He_two_body_integrand, "N_points":N_points}
integrals_num = lib.integral_master(N_basis)
integrals_num.calculate("integrals_He_num.npy", analytical_1, analytical_2=None, MC_args=MC_args)

# One- and Two-body integrals (analytical)
integrals_ana = lib.integral_master(N_basis)
integrals_ana.calculate("integrals_He_ana.npy", analytical_1, analytical_2=analytical_2)

We calculate the overlap matrix S

In [11]:
S = np.zeros((4,4))
for p in range(4):
    for q in range(4):
        S[p][q] = (np.pi/(ALPHA[p]+ALPHA[q]))**(1.5)
SVAL, SVEC = np.linalg.eigh(S) 
SVAL_minhalf = (np.diag(SVAL**(-0.5))) 
X = np.dot(SVEC, np.dot(SVAL_minhalf, np.transpose(SVEC)))
print('S = \n',S)

S = 
 [[1.20975063e+01 2.91187719e+00 3.71330014e-01 2.30637530e-02]
 [2.91187719e+00 1.42134692e+00 2.99025043e-01 2.22459699e-02]
 [3.71330014e-01 2.99025043e-01 1.41564997e-01 1.89120383e-02]
 [2.30637530e-02 2.22459699e-02 1.89120383e-02 8.24921040e-03]]


### Self-Consistent Field (numerical integration)

Finally, we run the Self-Consistent Field for Hartree Fock

In [16]:
lib.SCF(N_electrons, integrals_num, S, max_iter_SCF, eps_SCF)

E = -3.6343792 | N(SCF) = 1
E = -2.5614466 | N(SCF) = 2
E = -3.0212775 | N(SCF) = 3
E = -2.6325943 | N(SCF) = 4
E = -2.5879303 | N(SCF) = 5
E = -3.1690856 | N(SCF) = 6
E = -2.4971358 | N(SCF) = 7
E = -3.0378924 | N(SCF) = 8
E = -2.6284087 | N(SCF) = 9
E = -2.8876348 | N(SCF) = 10
E = -2.8382073 | N(SCF) = 11
E = -2.9325290 | N(SCF) = 12
E = -2.7595404 | N(SCF) = 13
E = -2.8659875 | N(SCF) = 14
E = -2.8495742 | N(SCF) = 15
E = -2.8758499 | N(SCF) = 16
E = -2.8168598 | N(SCF) = 17
E = -2.8468797 | N(SCF) = 18
E = -2.9038903 | N(SCF) = 19
E = -2.7148844 | N(SCF) = 20
E = -2.7992944 | N(SCF) = 21
E = -2.8873182 | N(SCF) = 22
E = -2.8053487 | N(SCF) = 23
E = -2.8208815 | N(SCF) = 24
E = -2.9050895 | N(SCF) = 25
E = -2.6797657 | N(SCF) = 26
E = -3.0375419 | N(SCF) = 27
E = -2.5755471 | N(SCF) = 28
E = -2.9566832 | N(SCF) = 29
E = -2.7657156 | N(SCF) = 30
E = -2.8687349 | N(SCF) = 31
E = -2.8809179 | N(SCF) = 32
E = -2.8813919 | N(SCF) = 33
E = -2.7992513 | N(SCF) = 34
E = -2.9296286 | N(SCF)

### Self-Consistent Field (analytical integration)

Finally, we run the Self-Consistent Field for Hartree Fock

In [13]:
lib.SCF(N_electrons, integrals_ana, S, max_iter_SCF, eps_SCF)

E = -2.0740068 | N(SCF) = 1
E = -2.9814045 | N(SCF) = 2
E = -2.9022846 | N(SCF) = 3
E = -2.8987992 | N(SCF) = 4
E = -2.7022179 | N(SCF) = 5
E = -3.0240475 | N(SCF) = 6
E = -2.4579730 | N(SCF) = 7
E = -2.6371765 | N(SCF) = 8
E = -2.9134126 | N(SCF) = 9
E = -2.8054441 | N(SCF) = 10
E = -2.8718781 | N(SCF) = 11
E = -2.8054162 | N(SCF) = 12
E = -2.9621827 | N(SCF) = 13
E = -2.7119546 | N(SCF) = 14
E = -2.8375567 | N(SCF) = 15
E = -2.9089723 | N(SCF) = 16
E = -2.8566725 | N(SCF) = 17
E = -2.8551020 | N(SCF) = 18
E = -2.8551718 | N(SCF) = 19
E = -2.8553565 | N(SCF) = 20
E = -2.8544635 | N(SCF) = 21
E = -2.8557063 | N(SCF) = 22
E = -2.8552542 | N(SCF) = 23
E = -2.8549395 | N(SCF) = 24
E = -2.8556334 | N(SCF) = 25
E = -2.8545766 | N(SCF) = 26
E = -2.8559940 | N(SCF) = 27
E = -2.8523017 | N(SCF) = 28
E = -2.8583181 | N(SCF) = 29
E = -2.8508570 | N(SCF) = 30
E = -2.8649039 | N(SCF) = 31
E = -2.8290428 | N(SCF) = 32
E = -2.8703410 | N(SCF) = 33
E = -2.8285136 | N(SCF) = 34
E = -2.8645429 | N(SCF)